LOADING PACKAGES

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import pandas as pd
import geopy
import geopy.distance
import numpy as np
import shapely.vectorized

LOADING DATA

In [ ]:
geo = pd.read_csv('data/geo_df.csv')
data = pd.read_csv('data/document_ratings.csv')

MODIFYING THE DATA

In [ ]:
# Filtering for countries of high confidence
geo = geo[geo['country_conf'] > 0.8]
geo = geo[pd.notnull(geo['country_predicted'])]
geo.lat = geo.lat.astype(float)
geo.lon = geo.lon.astype(float)

# Creating the main dataset
data_geo = pd.merge(data,geo,left_on="id",right_on="doc_id")
indexNames = data_geo[(data_geo['seen'] == 1) & (data_geo['relevant']==0)].index
data_geo.drop(indexNames, inplace=True)

columns_with_bad_names = [c for c  in data_geo.columns if "hidden" in c]
columns_with_bad_names.append("4 - Behavioral interventions")
data_geo.drop(columns_with_bad_names, 1, inplace=True)

data_geo.reset_index(drop=True, inplace=True)

CREATING THE FUNCTIONS REQUIRED TO PLOT THE DATA: TAKEN FROM MAX

In [ ]:
# Function to calculate great distance between two points on Earth
def new_haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1[:,None]
    dlat = lat2 - lat1[:,None]

    a = np.sin(dlat/2.0)**2 + np.cos(lat1[:,None]) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    
    return km

# Function to calculate the density grid for plotting

def density_grid(degrees,distance,df):
    
    df_countries = df[df["feature_code"]=="PCLI"]
    df_places = df[df["feature_code"]!="PCLI"]

    # linspace(start, stop, number of evenly spaced numbers)
    latbins = np.linspace(-90,90, round(180/degrees))
    lonbins = np.linspace(-180,180, round(360/degrees))

    n = np.zeros((len(latbins),len(lonbins)))
    
    print(f"calculating density grid of size: {n.size}")

    for i,lat in enumerate(latbins):
        # Calculating the geodesic distance between two points
        r = geopy.distance.distance(kilometers=distance)
        latp = geopy.Point((lat,135))
        # if the latitude is closer than distance to the north pole, then the northern bound should be 
        # the north pole, not distancekm north of the latitude (which will pass the pole and go south again)
        if geopy.distance.great_circle(latp,(90,135)).km < distance:
            r_nbound = 90   
        else:
            r_nbound = r.destination(point=latp,bearing=0).latitude
        # Same as above for the south pole
        if geopy.distance.great_circle(latp,(-90,135)).km < distance:
            r_sbound = -90   
        else:
            r_sbound = r.destination(point=latp,bearing=180).latitude        

        latbound_df = df_places[
            (df_places.lat>=r_sbound) &
            (df_places.lat<=r_nbound)        
        ]

        ds = new_haversine_np(latbound_df['lon'], latbound_df['lat'],lonbins,[lat]*len(lonbins))

        n[i,:] = np.where(ds<distance,1,0).sum(axis=0)
        
    print("done")
    
    shpfilename = shpreader.natural_earth(resolution='110m',
                                          category='cultural',
                                          name='admin_0_countries')
    reader = shpreader.Reader(shpfilename)
    yv, xv = np.meshgrid(latbins, lonbins)

    for country in reader.records():
        incountry = shapely.vectorized.contains(country.geometry,xv,yv)
        idx = np.argwhere(incountry==True)
        ndots = idx.size/2
        cdf = df_countries[df_countries["country_predicted"]==country.attributes["SU_A3"]]
        for point in idx:
            n[point[1],point[0]] += cdf.shape[0]/ndots

    return latbins, lonbins, n

FUNCTIONS TO SELECT THE DATA TO PLOT

In [ ]:
def create_list_of_predicted_categories(df):
    """Creates a list with all categories in the dataframe and a list with categories of the dataframe that have predictions.

    Args:
        df (DataFrame): The input dataframe (combined document ratings and geo data).

    Returns:
        all_cats: A list with all categories in the dataframe, as they are (including the "<hidden>" labels that are found in some columns).
        pred_cats: A list with categories that have predictions, as they are (including the "<hidden>" labels that are found in some columns).
    """
    data = df.copy()
    
    all_cats = [c for c  in data.columns if "-" in c and "prediction" not in c] # This list contains all variable columns.
    preds = [c for c in data.columns if "prediction" in c] # This list contains all columns that are predictions.
    pred_cats = []
    for c in all_cats:
        for p in preds:
            if c in p:
                pred_cats.append(c)
    return all_cats, pred_cats

In [ ]:
def merge_columns(df, col, col_pred):
    """Creates a dataframe with a merged human/prediction column.

    Args:
        df (DataFrame): The input dataframe (combined document ratings and geo data).
        col (str): Name of the column that contains the variable.
        col_pred (str): Name of the column with the predicted variable.

    Returns:
        A modified dataframe with a column that has the predicted values if the paper was not seen by a human, and the "human" values if the paper was seen. The column has the same name of the variable.
    """
    
    data = df.copy()
    data['human'] = data[col] # This creates a copy of human data under the "human" header
    data[col] = data[col_pred] # This replaces the column of interest (human data) with predicted data
    
    i = 0
    while(i < len(data)):
        if data['seen'].iloc[i] == 1:
            data.at[i, col] = data['human'].iloc[i] # This replaces the predicted data, which is under the "column of interest" header, with the human data from the copied column.
        i = i + 1

    return data

In [ ]:
def filter(df, cat, prob=0.5):
    """Filters a dataframe based on a variable of interest.

    Args:
        df (DataFrame): The input dataframe (combined document ratings and geo data).
        cat (str): Name of the column that contains the variable. The name should be precise, including "<hidden>" labels. 'All' if no filtering is to be done.
        prob (float): Minimum probability threshold to consider that a paper is about the variable of interest. Default: 0.5

    Returns:
        A dataframe that has been filtered according to a specific variable.
    """
    filtered = df.copy()
    all_cats, pred_cats = create_list_of_predicted_categories(df)
    
    if(cat == 'All' or cat == 'all'):
        return filtered
    elif(cat in pred_cats):
        pred = cat + ' - prediction'
        filtered = merge_columns(filtered, cat, pred)
        filtered = filtered[filtered[cat] >= prob]
        return filtered
    elif(cat in all_cats):
        filtered = filtered[filtered[cat] >= prob]
        return filtered
    else:
        print('Wrong variable')

FUNCTIONS TO PLOT

In [ ]:
def plot_map(cat, degrees, distance, df, file_name, dpi=150, width=8, height=5):
    """Creates a map based on a variable of interest and saves it in a map.

    Args:
        cat (str): Variable of interest. Must be specific, including "<hidden>" labels. "All" if all papers are to be mapped.
        degrees (float): Number of degrees.
        distance (int): Distance of the clusters.
        df (DataFrame): The input dataframe (combined document ratings and geo data).
        file_name (str): Name of the plot file.
        dpi (int): Resolution (dots per inch).
        width (int): Width of the plot. Default: 8.
        height (int): Height of the plot. Default: 5.

    Returns:
    """
    fig, ax = plt.subplots(dpi=dpi, figsize=(width, height))
    p = ccrs.Mollweide()
    ax = plt.axes(projection=p)
    ax.set_global()
    ax.coastlines(lw=0.1)

    filtered = filter(df, cat)
    latbins, lonbins, n = density_grid(degrees,distance,filtered)

    vm = n[~np.isnan(n)].max()
    n[n == 0] = np.nan

    pcm = plt.pcolormesh( 
        lonbins, latbins, n,
        transform=ccrs.PlateCarree(),
        norm=mpl.colors.LogNorm(vmin=1, vmax=vm),
        alpha=0.5,
        cmap="YlGnBu"
    )

    fig.colorbar(pcm)
    title = cat.split(" - ")[1]
    ax.set_title(title)
    filename = 'plots/' + file_name + '.pdf'
    plt.savefig(filename)

In [ ]:
def plot_maps(df):
    """Plots all variables in a dataframe.

    Args:
        df (DataFrame): The input dataframe (combined document ratings and geo data).

    Returns:
    """
    all_cats, pred_cats = create_list_of_predicted_categories(df)
    for counter, c in enumerate(all_cats):
        file_name = c.split(' - ')[1].replace(" ","_").replace("/", "_")
        print(file_name)
        plot_map(c, 1, 100, df, file_name)
        print(str((counter/len(all_cats))*100) + '%')

In [ ]:
plot_maps(data_geo)